## test find

In [ ]:
# test find(e,element_list)

import random
# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
# ==========================data structure==========================

def find(e,element_list):
    try:
        index = [ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index

filename='kosarak_ground_truth.csv'
filepath="..\\result\\kosarak\\"
src_data=os.path.join(filepath,filename)

df=pd.read_csv(src_data)
x=[Tail(i,j) for i,j in df.values.tolist()]
y=[]
randomindex=[random.randint(1,len(x)) for i in range(100)]
for index in randomindex:
    y.append(x[index])
for item in y:
    result=find(item,x)
    print("item in y: {},find index:{},item in x[{}]:{}".format(item,result,result,x[result]))
    print("item?=x[result]:{}".format(item==x[result]))

## UpdateSK

In [ ]:
import spookyhash
import mmh3
from numpy import random
import os
import time
import operator
import hyperloglog

# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}, max: {}]'.format(self.count,len(self.distinct),self.maxID)
    def __repr__(self):
        return '[total count: {}, distinct: {}, max: {}]'.format(self.count,len(self.distinct),self.maxID)

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
# ==========================data structure==========================


# ==========================UpdateSk==========================
def UpdateSk(element,width,depth):
    e_max=get_emax()
    hash1=spookyhash.hash32(bytes(str(element.ID),encoding='utf-8'))
        # input of spooky hash: binary, encoding is parameter of bytes()
        # output of spooky hash: unsigned- 32 bit int
    hash2=mmh3.hash(str(hash1), signed=False)
        # input of mmh3 is str, output unsigned- 32 bit int
    row=hash1 % depth
        # index of row in SK[row]
    ID=hash2 % ((width*3)//2)
        # hash-value(ID) of e
        # range: 2^k *3/2= 3*(2^k-1)
    match=False
        # match= True if e is in Sk[row]
    index=0
    avg=0
    item=Tail(ID, element.count)
        # ID in Sketch is hash value
    print("{} -> {},send to Sk[{}]".format(element,item,row))

    # ==========================update sketch==========================
    index=find(item,Sk[row])
        # 先find
    if index >=0:
        # e matches in Sk[row]
        Sk[row][index].count+=item.count
        match=True
    else:
        # e doesn't match in Sk[row]
        if len(Sk[row])==0:
            Sk[row].append(item)
            index=0
            Sk_head[row].maxID=element.ID
            match=True
        elif len(Sk[row])<width:
            # Sk[row] is not full
            Sk[row].append(item)
            index=len(Sk[row])-1            
            match=True
        else:
            # Sk[row] is full
            count_sum=sum(Sk[row][i].count for i in range(len(Sk[row])))
            if Sk_head[row].count==count_sum:
                # no other, send e into Other
                Sk_head[row].distinct.add(element.ID)
                match=False
            else:
                # something in other and total count > sum
                if len(Sk_head[row].distinct)>0:
                    avg=(Sk_head[row].count-count_sum)//len(Sk_head[row].distinct)
                    print("avg:{}".format(avg))
                else:
                    Sk_head[row].distinct.add(element.ID)
                    # print("avg< last one,send to distinct")
                    match=False
    Sk_head[row].count+=item.count
        # total count最後再+1 因為要compare distinct
    # ==========================update local max==========================
    Update_local_max(Sk_head[row],Sk[row],avg,element,index)
    # Sk[row].sort(key=operator.attrgetter('count'),reverse=True)

    # ==========================update e_max==========================
    Update_emax(Sk_head[row],Sk[row],e_max,element)
    
    Sk[row].sort(key=operator.attrgetter('count'),reverse=True)
    for i in range(len(Sk)):
        print("Sk[{}]:{},{}".format(i,Sk_head[i],Sk[i]))
# ==========================update local max==========================    
def Update_local_max(head_item,element_list,avg,incoming_element,index):
    # Sk_head[row],Sk[row],avg
    print("In Update_local_max")
    local_max_ID=mmh3.hash(str(spookyhash.hash32(bytes(str(head_item.maxID),encoding='utf-8'))), signed=False) % ((width*3)//2)
    #print("local_max_ID:{}".format(local_max_ID))
    local_max_index=find(Tail(local_max_ID,1),element_list)
    #print("local_max_index:{}".format(local_max_index))
    # 此處當hashed ID相同時暫不更新 local max
    if local_max_index>=0:
        # local max exists
        if index>=0:
            if element_list[index].count>element_list[local_max_index].count:
                head_item.maxID=incoming_element.ID
        else:
            if avg>element_list[local_max_index].count:
                head_item.maxID=incoming_element.ID
    else:
        # local max doesn't exists
        head_item.maxID=incoming_element.ID
    

# ==========================update e_max==========================
def Update_emax(head_item,element_list,e_max,incoming_element):
    # Sk_head[row],Sk[row],e_max
    local_max_ID=""
    local_max_index=""
    print("e_max before update:{}".format(e_max))
    print("Sk_head:{}".format(head_item))
    print("Sk[row]:{}".format(element_list))
    if head_item.maxID=="":
        pass
    else:
        local_max_ID=mmh3.hash(str(spookyhash.hash32(bytes(str(head_item.maxID),encoding='utf-8'))), signed=False) % ((width*3)//2)
        #print("local_max_ID:{}".format(local_max_ID))
        local_max_index=find(Tail(local_max_ID,1),element_list)        
        #print("local_max_index:{}".format(local_max_index))
        if local_max_index>=0:
            if element_list[local_max_index].count>e_max.count:
                e_max.ID=head_item.maxID
                e_max.count=element_list[local_max_index].count
        else:
            pass
    print("e_max after update:{}".format(e_max))

# ==========================Tools=========================    
def get_emax():
    return e_max

def find(e,element_list):
    try:
        # type(element_list[i].ID):int
        # print("e:{}".format(e))
        # print("element_list[1:]: {}".format(element_list[1:]))
        index=[ele.ID for ele in element_list].index(e.ID)
    except:
        # print("raise error")
        index=-99
    return index                        
# ==========================Tools=========================   
        
start=time.time()

filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)
depth=4
width=4
size=4

Top=[]
Sk_head=[Head(0) for j in range(depth)]
Sk=[[] for i in range(depth)]
item_count=70
i=0
e_max=Tail('',0)

with open(src_data,'r') as file:
    while item_count:
        element=file.readline().strip('\n')
        if not element:
            break
        else:
            i+=1
            print("\nread {}-th element: {}".format(i,element))
            item_count-=1
            item=Tail(element,1)
            if len(Top)==0:
                Top.append(item)
                # print("len(Top)==0, Top append:{}".format(Top))
            else:
                index=find(item,Top)
                # print("index={}".format(index))
                if index<0:
                    if len(Top)<size:
                        Top.append(item)
                        # print("index={},Top after append: {},\nlen(Top):{}".format(index,Top,len(Top)))
                    else:
                        UpdateSk(item,width,depth)
                else:
                    # print("update Top[{}]:".format(index))
                    Top[index].count+=1
            # print("Top after read {}:\n{}\n".format(element,Top))
            Top.sort(key=operator.attrgetter('count'),reverse=True)
            '''
            if e_max.count>Top[-1].count:
                BringBack(Top[-1],e_max)
                # print('Top after BringBack: \n\t{}'.format(Top))            
            # print("TOP:{}".format(Top))            
            '''


end=time.time()

print("e_max:{}".format(e_max))
print("Execution time:{} seconds.".format(str(end-start)))

## DeleteSK

In [ ]:


def DeleteSk(element):
    # Delete element in SK and update e_max
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
    hash2=mmh3.hash(str(hash1), signed=False)
    ID=hash2 % ((width*3)//2)
    row=hash1 % depth
    Sk[row][0].count-=element.count
    index=find(ID,Sk[row])
    if index<0:
        pass
    else:
        Sk[row].pop(index)


filename='kosarak_ground_truth.csv'
filepath="..\\result\\kosarak\\"
src_data=os.path.join(filepath,filename)
df=pd.read_csv(src_data)
x=[Tail(i,j) for i,j in df.values.tolist()]

depth=4
width=128
Sk=[[ds.Head(0)] for j in range(depth)]


In [ ]:

import copy

x=copy.copy(Sk[3])
print(id(x),id(Sk[3]))
print(x)

def sort_Sk(element_list):
    tail=element_list[1:]
    tail.sort(key=operator.attrgetter('count'),reverse=True)
    return [element_list[0]]+tail
    
Sk[3]=sort_Sk(x)
print(Sk[3])